In [15]:
import getpass
import os
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate

os.environ['OPENAI_API_KEY'] ="KEY"

model = ChatOpenAI(model="gpt-4-turbo-preview")

prompt = ChatPromptTemplate.from_template("tell me a short joke about {topic}")
output_parser = StrOutputParser()

chain = prompt | model | output_parser

chain.invoke({"topic": "ice cream"})



'Why did the ice cream truck break down?\n\nBecause of the rocky road!'

In [8]:
prompt_value = prompt.invoke({"topic": "ice cream"})
prompt_value

ChatPromptValue(messages=[HumanMessage(content='tell me a short joke about ice cream')])

In [9]:
prompt_value.to_messages()

[HumanMessage(content='tell me a short joke about ice cream')]

In [10]:
prompt_value.to_string()


'Human: tell me a short joke about ice cream'

In [11]:
message = model.invoke(prompt_value)
message

AIMessage(content="Why don't ice creams ever get invited to parties?\n\nBecause they always drip when they're getting ready!", response_metadata={'token_usage': {'completion_tokens': 21, 'prompt_tokens': 15, 'total_tokens': 36}, 'model_name': 'gpt-4', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-6b949c8c-55f6-4134-8111-d0a2a2785e0d-0')

In [18]:
from langchain_openai import OpenAI

llm = OpenAI(model="gpt-3.5-turbo-instruct")
llm.invoke(prompt_value)

'\n\nRobot: Why was the ice cream feeling sad? Because it was having a rocky road!'

In [13]:
output_parser.invoke(message)

"Why don't ice creams ever get invited to parties?\n\nBecause they always drip when they're getting ready!"

In [16]:
input = {"topic": "ice cream"}

prompt.invoke(input)
# > ChatPromptValue(messages=[HumanMessage(content='tell me a short joke about ice cream')])

(prompt | model).invoke(input)
# > AIMessage(content="Why did the ice cream go to therapy?\nBecause it had too many toppings and couldn't cone-trol itself!")

AIMessage(content='Why did the ice cream truck break down? \n\nBecause of the rocky road!', response_metadata={'token_usage': {'completion_tokens': 16, 'prompt_tokens': 15, 'total_tokens': 31}, 'model_name': 'gpt-4-turbo-preview', 'system_fingerprint': 'fp_f38f4d6482', 'finish_reason': 'stop', 'logprobs': None}, id='run-7084593d-124f-4fa7-98f9-5de388ca5fd5-0')

RAG Search

In [23]:
# Requires:
# pip install langchain docarray tiktoken

from langchain_community.vectorstores import DocArrayInMemorySearch
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableParallel, RunnablePassthrough
from langchain_openai import OpenAIEmbeddings

vectorstore = DocArrayInMemorySearch.from_texts(
    ["harrison worked at kensho", "bears like to eat honey"],
    embedding=OpenAIEmbeddings(),
)
retriever = vectorstore.as_retriever()

template = """Answer the question based only on the following context:
{context}

Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)
output_parser = StrOutputParser()

setup_and_retrieval = RunnableParallel(
    {"context": retriever, "question": RunnablePassthrough()}
)
chain = setup_and_retrieval | prompt | model | output_parser



'Bears like to eat honey.'

In [24]:
chain.invoke("what is the date today?")

"I'm sorry, but I cannot provide the current date as my responses are based on the information provided in the given context, which does not include any information about the current date."

In [21]:
retriever.invoke("where did harrison work?")

[Document(page_content='harrison worked at kensho'),
 Document(page_content='bears like to eat honey')]